In [1]:
import seaborn as sns

In [2]:
#import all useful modules
import numpy as np
import pandas as pd
from numpy import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import integrate
import astropy.units as u
import astropy.constants as c
import kepler
pd.set_option('display.max_columns', None)
from scipy.integrate import quad

In [3]:
exoplanet_data = pd.read_csv('/Users/Kristinagatto/Desktop/kepler_paper.csv')

'''
You must upload your data as a csv file and set it to the variable exoplanet data
'''

'\nYou must upload your data as a csv file and set it to the variable exoplanet data\n'

In [4]:
# Name Variables
pi = np.pi
G = 6.6743e-11  #* (10**(-11)) #gravitational constant
year_sec = 3.154e7 #*10**7
half_year = year_sec / 2
jupiter_mass = 1.89e27 #*10**27
sun_mass = 1.989e30 #*10**30 #solar mass
earth_mass = sun_mass/333000 #kg
jupit_mass = 1#.898 * 10**27
sun_mass_jup = 1047#*jupit_mass

#define all functions needed

def semi_amplitude(period,planet_mass,star_mass,eccentricity,inclination):
    '''
    This function takes in the period of the orbiting exoplanet, 
    the mass of both the exoplanet and the star, the eccentricity of the
    orbiting exoplanet, and inclination and returns the semi-amplitude of 
    the exoplanet.
    
    '''
    #Variables
    pi = np.pi
    G = 6.6743e-11
    ##############
    
    one = (((2*pi*G)/period)**(1/3))
    two = ((planet_mass)*(np.sin(inclination))) / ((star_mass + planet_mass)**(2/3))
    three = 1/(np.sqrt(1-(eccentricity**2)))
    
    sem_amp = one*two*three
    return sem_amp

def RadialVelocity1(m_1,q,e,i,a,phi,w,t,eps):
    m_2=m_1*q
   
    
    period = 2*np.pi*np.sqrt((a*c.au)**3/(c.G*(m_1+m_2)*c.M_sun))
    n = 2*np.pi/period.values
    
    
    mean_anomaly = n*(t)+phi
    eccentric_anomaly, cos_true_anomaly, sin_true_anomaly = kepler.kepler(mean_anomaly, e)
    k = np.sqrt(c.G/(((m_1+m_2)*c.M_sun)*(a*c.au)*(1-e**2)))*(m_2*c.M_sun)*np.sin(np.deg2rad(i))
    b = (np.cos(w) * cos_true_anomaly - np.sin(w) * sin_true_anomaly + e * np.cos(w))
    
    rv = k*b
    #print(rv)
    RV = (rv).values
    #sigma = np.array(np.random.uniform(low=2, high=3,size=1), dtype=float)
    noisy_RV = RV+(eps*np.random.randn(*RV.shape))
    #print(period/86400)
    return noisy_RV,k,b

def f_value(a1,a2):
    psi = np.linspace(0,2*pi,num=100000,endpoint=True)
    funct = (np.cos(psi)) / ((1 - (2*(a1/a2))*(np.cos(psi))+((a1/a2)**2))**(3/2))
                             
    fvalue = lambda psi: funct
    f = np.trapz(funct)
    
    return f 

def f2_value(a1,a2):
    psi = np.linspace(0,2*pi,num=100000,endpoint=True)
    funct = (np.cos(2*psi)) / ((1 - (2*(a1/a2))*(np.cos(psi))+((a1/a2)**2))**(3/2))
                             
    fvalue = lambda psi: funct
    f = np.trapz(funct)
    
    return f   

def f_sci(psi, a1, a2):

    return (np.cos(psi)) / ((1 - (2*(a1/a2))*(np.cos(psi))+((a1/a2)**2))**(3/2))

def f2_sci(psi, a1, a2):

    return (np.cos(2*psi)) / ((1 - (2*(a1/a2))*(np.cos(psi))+((a1/a2)**2))**(3/2))

def a_22(m1,m2,big_m,a1,a2,f):
    T = np.sqrt(4*(pi**2)*((a2*1.496E11)**3)/(G*((big_m*(1.89)*10E27)+(m2*(1.89)*10E27))))
    n_2 = 2*pi/T
    one = (n_2) * (1/(4*pi))
    two = m1 / (big_m + m2)
    three = ((a1/a2)**2) * f
    
    a22 = one*two*three
    return a22

def a_21(m1,m2,big_m,a1,a2,f):
    T = np.sqrt(4*(pi**2)*((a2*1.496E11)**3)/(G*((big_m*(1.89)*10E27)+(m2*(1.89)*10E27))))
    n_2 = 2*pi/T    
    one = (-n_2) * (1/(4*pi))
    two = m1 / (big_m + m2)
    three = (a1/a2) * f
    
    a21 = one*two*three
    return a21

def b_22(m1,m2,big_m,a1,a2,f):
    T = np.sqrt(4*(pi**2)*((a2*1.496E11)**3)/(G*((big_m*(1.89)*10E27)+(m2*(1.89)*10E27))))
    n_2 = 2*pi/T    
    one = (-n_2) * (1/(4*pi))
    two = m1 / (big_m + m2)
    three = ((a1/a2)**2) * f
    
    b22 = one*two*three
    return b22

def b_21(m1,m2,big_m,a1,a2,f):
    T = np.sqrt(4*(pi**2)*((a2*1.496E11)**3)/(G*((big_m*(1.89)*10E27)+(m2*(1.89)*10E27))))
    n_2 = 2*pi/T    
    one = (n_2) * (1/(4*pi))
    two = m1 / (big_m + m2)
    
    three = (a1/a2) * f
    b21 = one*two*three
    return b21

def f_ll(a22,a21,e1ic,e2ic,omega1,omega2,b21,i1ic,i2ic,big_omega1,big_omega2,b22):
    
    one = a21*(e1ic/e2ic)*(np.cos((omega1-omega2)))
    two = b21*(i1ic/i2ic)*(np.cos(big_omega1-big_omega2))
    #print(one)
    #print(two)
    answer = a22 + one - two - b22
    return answer

def f_llMin(a22,a21,e1ic,e2ic,b21,i1ic,i2ic,b22):
    
    one = a21*(e1ic/e2ic)
    two = b21*(i1ic/i2ic)
    #print(one)
    #print(two)
    answer = a22 + one - two - b22
    return answer

def f_llMax(a22,a21,e1ic,e2ic,b21,i1ic,i2ic,b22):
    
    one = a21*(e1ic/e2ic)
    two = b21*(i1ic/i2ic)
    #print(one)
    #print(two)
    answer = a22 - one - two - b22
    return answer

def e3_c(big_m,m1,m2,m3,a2,a3,fll):
    G = 6.6743*10**-11  #gravitational constant m^3 kg^-1 s^-2
    
    one = 15/16
    two = ((m3*(1.89)*10E27)*(np.sqrt(G)))/(np.sqrt((big_m*(1.89)*10E27) + (m1*(1.89)*10E27) + (m2*(1.89)*10E27)))
    three = ((a2*1.496E11)**(3/2))/((a3*1.496E11)**3)
    four = np.abs(1/fll)
    
    inside = (one*two*three*four)**(2/3)
   
    e3c = np.sqrt(1 - inside)
    #print(inside)
    return e3c

In [5]:
#Simulation function

def simulations(exoplanet_data):

    ecc1 = exoplanet_data['ecc_planet_1']  # if there are no eccentricity values just use .01
    ecc2 = exoplanet_data['ecc_planet_2']  
    mass1 = exoplanet_data['mass_planet_1'] # / 317.907		####MUST CONVERT TO JUPITER MASSES
    mass2 = exoplanet_data['mass_planet_2'] #/ 317.907
    a1value = exoplanet_data['au_planet_1']
    a2value = exoplanet_data['au_planet_2']
    name = f"{exoplanet_data['name']}" #Exoplanet name
    
    N=1000000
    test_df=pd.DataFrame(columns=['big_m','e1ic','e2ic','i1ic','i2ic','omega1','omega2','big_omega1','big_omega2','m1','m2','m3','a1','a2','a3'])
    test_df['big_m'] = np.full_like(random.uniform(10,1000,(N)),sun_mass_jup,dtype=float).tolist()
    test_df['e1ic'] = np.full_like(random.uniform(10,1000,(N)),ecc1,dtype=float).tolist()
    test_df['e2ic'] = np.full_like(random.uniform(10,1000,(N)),ecc2,dtype=float).tolist()
    test_df['i1ic'] = np.full_like(random.uniform(10,1000,(N)),pi/2,dtype=float).tolist()
    test_df['i2ic'] = np.full_like(random.uniform(10,1000,(N)),pi/2,dtype=float).tolist()
    test_df['omega1'] = np.full_like(random.uniform(10,1000,(N)),pi,dtype=float).tolist()
    test_df['omega2'] = np.full_like(random.uniform(10,1000,(N)),pi+.1,dtype=float).tolist()
    test_df['big_omega1'] = np.full_like(random.uniform(10,1000,(N)),pi+.1).tolist()
    test_df['big_omega2'] = np.full_like(random.uniform(10,1000,(N)),pi).tolist()
    test_df['m1'] = np.full_like(random.uniform(10,1000,(N)),mass1).tolist()
    test_df['m2'] = np.full_like(random.uniform(10,1000,(N)),mass2).tolist()
    #test_df['m3'] = np.random.choice([10,50,150,250,500],N,).tolist()
    #test_df['m3'] = np.random.choice([10,50,100,150,200],N,).tolist()
    test_df['m3'] = np.random.choice([.5,1,10,20,50],N,).tolist()
    #test_df['m3'] = np.random.choice([.1,.5,1,5,20],N,).tolist()
    test_df['a1'] = np.full_like(random.uniform(10,1000,(N)),a1value).tolist()
    test_df['a2'] = np.full_like(random.uniform(10,1000,(N)),a2value).tolist()
    test_df['a3'] = random.uniform(0,80,(N)).tolist()
    

    test_df['f1'] = quad(f_sci, 0, 2*pi, args=(test_df['a1'].loc[1],test_df['a2'].loc[1]))[0]
    test_df['f2'] = quad(f2_sci, 0, 2*pi, args=(test_df['a1'].loc[1],test_df['a2'].loc[1]))[0]
    test_df['a22'] = a_22(test_df['m1'],test_df['m2'],test_df['big_m'],test_df['a1'],test_df['a2'],test_df['f1'])
    test_df['a21'] = a_21(test_df['m1'],test_df['m2'],test_df['big_m'],test_df['a1'],test_df['a2'],test_df['f2'])
    test_df['b22'] = b_22(test_df['m1'],test_df['m2'],test_df['big_m'],test_df['a1'],test_df['a2'],test_df['f1'])
    test_df['b21'] = b_21(test_df['m1'],test_df['m2'],test_df['big_m'],test_df['a1'],test_df['a2'],test_df['f1'])
    test_df['fll'] = f_ll(test_df['a22'],test_df['a21'],test_df['e1ic'],test_df['e2ic'],test_df['omega1'],test_df['omega2'],test_df['b21'],test_df['i1ic'],test_df['i2ic'],test_df['big_omega1'],test_df['big_omega2'],test_df['b22'])
    test_df['fll_min'] = f_llMin(test_df['a22'],test_df['a21'],test_df['e1ic'],test_df['e2ic'],test_df['b21'],test_df['i1ic'],test_df['i2ic'],test_df['b22'])
    test_df['fll_max'] = f_llMax(test_df['a22'],test_df['a21'],test_df['e1ic'],test_df['e2ic'],test_df['b21'],test_df['i1ic'],test_df['i2ic'],test_df['b22'])

    test_df['e3c'] = e3_c(test_df['big_m'],test_df['m1'],test_df['m2'],test_df['m3'],test_df['a2'],test_df['a3'],test_df['fll'])
    test_df['e3c_min'] = e3_c(test_df['big_m'],test_df['m1'],test_df['m2'],test_df['m3'],test_df['a2'],test_df['a3'],test_df['fll_min'])

    test_df['e3c_max'] = e3_c(test_df['big_m'],test_df['m1'],test_df['m2'],test_df['m3'],test_df['a2'],test_df['a3'],test_df['fll_max'])
    test_df['sem_amp']=RadialVelocity1(1,test_df['m3']/1047,0,90,test_df['a3'],0,0,10,.1)[1]

    return test_df

'''
   This function actually creates the simulations. The data you input must be
   in a certain format, otherwise it will not run. The data required to create these
   simulations is a two-planet system, each planet's eccentricity (ecc1, ecc2), 
   each planet's mass (mass1, mass2) IN JUPITER MASSES (THIS IS CRUCIAL), the 
   separation (A1, A2) distance each planet is from the main star in AU, and the planet's name.
   You must have a .csv file with columns named name, ecc_planet_1, ecc_planet_2, mass_planet_1, 
   mass_planet_2, au_planet_1, au_planet_2, and name. If there are no valid eccentricity values 
   available for certain planetary systems, simply use 0.01 for any and all.
   '''

"\n   This function actually creates the simulations. The data you input must be\n   in a certain format, otherwise it will not run. The data required to create these\n   simulations is a two-planet system, each planet's eccentricity (ecc1, ecc2), \n   each planet's mass (mass1, mass2) IN JUPITER MASSES (THIS IS CRUCIAL), the \n   separation (A1, A2) distance each planet is from the main star in AU, and the planet's name.\n   You must have a .csv file with columns named name, ecc_planet_1, ecc_planet_2, mass_planet_1, \n   mass_planet_2, au_planet_1, au_planet_2, and name. If there are no valid eccentricity values \n   available for certain planetary systems, simply use 0.01 for any and all.\n   "

In [6]:
#Run simulations

def exocompanions(exoplanet_data): #IDK IF I NEED THIS
    
    for i in exoplanet_data['name']:
        
        exosim = simulations(exoplanet_data[exoplanet_data['name']==i])
        #print(exosim)
        sns.set_style("ticks")
        plt.rcParams['figure.figsize'] = [16, 12]
        plt.rcParams['font.family'] = "serif"
        plt.rcParams["errorbar.capsize"] = 1

        params = {'legend.fontsize': 'xx-large',
                 'axes.labelsize': 'xx-large',
                 'axes.titlesize':'xx-large',
                 'xtick.labelsize':'xx-large',
                 'ytick.labelsize':'xx-large'}


        plt.rcParams['mathtext.fontset'] = 'dejavuserif'
        plt.rcParams.update(params)
        plt.rc('font',**{'family':'serif','sans-serif':['Times']})
        plt.rcParams['font.family'] = "serif"
    
    
        plt.scatter(exosim['a3'],exosim['e3c_max'], c=RadialVelocity1(1,exosim['m3']/1047,exosim['e3c'],90,exosim['a3'],0,0,10,.1)[1]/1000, cmap='viridis')
        c = plt.colorbar(label='semi-amplitude')
        c.set_label('semi-amplitude',fontsize=25)
       
        #plt.clim(0, 15) #np.max(RadialVelocity1(1,exosim['m3']/1047,exosim['e3c'],90,exosim['a3'],0,0,10,.1)[1]/1000))
        plt.xlabel('Distance (AU)', size=25)
        plt.ylabel('Eccentricity',size=25)
        name = exoplanet_data['name'][exoplanet_data['name']==i].values
        #plt.title(f'KOI {name}',size=30)
        plt.title(f'{name}',size=30)
        #plt.xlim(0,exoplanet_data['au_planet_2'][exoplanet_data['name']==i].values+5)
        plt.ylim(0,1)
        plt.savefig(f'{name}.png')
        plt.close()

In [7]:
run = exocompanions(exoplanet_data)

/Users/kristinagatto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kristinagatto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kristinagatto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kristinagatto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kristinagatto/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kristinagatto